In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

In [2]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

In [3]:
index_name = "wiki"
for idx in pc.list_indexes():
    # 인덱스 이름이 "wiki"와 일치하는 경우 해당 인덱스를 삭제합니다.
    if idx.name == index_name:
        pc.delete_index(idx.name)

In [4]:
from pinecone import ServerlessSpec
pc.create_index(
    name='wiki', # 인덱스 이름
    dimension=1536,
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws', # 서버없이 
        region='us-east-1'
    )
)

{
    "name": "wiki",
    "metric": "cosine",
    "host": "wiki-ou15dnm.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [5]:
# 'wiki' 인덱스를 가져옵니다.
index = pc.Index(index_name)

index.describe_index_stats()

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

# data load

In [6]:
%pip install -qU datasets

Note: you may need to restart the kernel to use updated packages.


In [7]:
from datasets import load_dataset
data = load_dataset("wikimedia/wikipedia", "20231101.ko", split='train[:100]')

In [8]:
data

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 100
})

In [9]:
data[0]

{'id': '5',
 'url': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0',
 'title': '지미 카터',
 'text': '제임스 얼 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n\n정계 입문 \n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n\n대통령 재임 \n\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받았는데 제럴드 포드 대통령을 누르고 당선되었다.\n\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n\n외교 정책 \n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으

In [10]:
len(data)

100

In [11]:
for rec in data:
    if len(rec['text']) > 35000: # 파이콘의 경우  20kbyte 초과하면 안됨
        print('**')

**
**


In [12]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small')
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000017FB69A2960>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000017FB85591C0>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter( # 큰 것부터 자름
    chunk_size = 400, # 글자를 400자 기준으로 자름.
    chunk_overlap = 20,
    length_function = len, \
    separators = ["\n\n", "\n", " ", ""] # 자르는 순서 문자 지정  
)
splitter

In [14]:
import json
def calculate_metadata_size(metadata):
    return len(json.dumps(metadata, ensure_ascii=False).encode('utf-8'))

In [15]:
from uuid import uuid4
import time

texts = []
metas = []
count = 0
batch_size = 20

In [16]:
for i, sample in enumerate(data):
    full_text = sample['text'] # 전문
    
    # 메타데이터 구성
    metadata = {
        "wiki_id":str(sample['id']),
        "url":sample['url'],
        "title":sample['title']  #[:100]
    }
    
    # 스를릿터로 자르기
    chunks = splitter.split_text(full_text)
    # print(len(chunk))
    
    for i, chunk in enumerate(chunks):
        record = {
            'chunk_id' : i,
            'full_text': full_text,
            **metadata
        }
        metadata_size = calculate_metadata_size(record)
        if metadata_size > 35000:
            continue
        
        texts.append(chunk)
        metas.append(record)
        count += 1
        
        if count % batch_size == 0:
            ids = [str(uuid4()) for _ in range(len(texts))]
            embeddings = embedding.embed_documents(texts) # 20개 청크의 임베딩 수행
            index.upsert(
                vectors = zip(ids, embeddings, metas),
                namespace='wiki-ns1'
            )
            texts=[]
            metas=[]
            time.sleep(1)

In [17]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wiki-ns1': {'vector_count': 1160}},
 'total_vector_count': 1160,
 'vector_type': 'dense'}

# 질의 임베딩

In [25]:
question = ["벨기에는 어디에 있나요?"]
emb_question = embedding.embed_documents(question)
emb_question

[[0.0029317522421479225,
  0.038531601428985596,
  0.0835522934794426,
  0.01172700896859169,
  -0.01034860871732235,
  -0.015395675785839558,
  0.011822436936199665,
  0.015851609408855438,
  -0.028034551069140434,
  -0.02203320525586605,
  -0.019255198538303375,
  0.05802006646990776,
  0.04222147539258003,
  -0.030833764001727104,
  0.0014698522863909602,
  0.0055931261740624905,
  -0.06930174678564072,
  0.01671045832335949,
  0.026719767600297928,
  0.02192717418074608,
  -0.023347988724708557,
  -0.022966276854276657,
  -0.032254576683044434,
  0.048646941781044006,
  0.042560774832963943,
  0.007125271484255791,
  0.01567135564982891,
  0.00029837735928595066,
  0.00684959115460515,
  -0.036156512796878815,
  0.022669389843940735,
  -0.0325726680457592,
  0.021344006061553955,
  -0.05038584768772125,
  0.014197527430951595,
  0.009839660488069057,
  0.045126721262931824,
  -0.0440664105117321,
  0.017866194248199463,
  -0.023560049012303352,
  -0.006059661507606506,
  0.00516370

In [26]:
# 파인콘 질의 실행
q_result = index.query(
    namespace='wiki-ns1',
    vector=emb_question,
    top_k=5,
    include_values=False,
    include_metadata=True
)
q_result

{'matches': [{'id': '7c5cc093-854b-4395-87d0-535acb11f190',
              'metadata': {'chunk_id': 11.0,
                           'full_text': '엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 '
                                        '없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. '
                                        '통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 '
                                        '로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, '
                                        '열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 '
                                        '루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 '
                                        '라틴 대문자 S를 쓴다.\n'
                                        '\n'
                                        '정의 \n'
                                        '엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 '
                                        '정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. '
                                        '고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 '
                                      

In [27]:
result_ids = [r.id for r in q_result.matches]
result_ids

['7c5cc093-854b-4395-87d0-535acb11f190',
 'e2f76e38-4923-4900-8b51-60f6e522c25d',
 '661164d2-11f3-4d79-9fa4-3b93e7dec8e1',
 '4f256005-fbd0-4521-aba8-f74932b3a561',
 'b27fc866-c528-42fe-917c-a9d1bbd1a6aa']

In [28]:
for r in q_result.matches:
    print(r.score, r.metadata)

0.306918174 {'chunk_id': 11.0, 'full_text': '엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. 통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, 열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 라틴 대문자 S를 쓴다.\n\n정의 \n엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. 고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 성분들로 이루어져 있고, 학설의 안정성은 그러한 성분들의 평균적인 열특성으로 설명된다.\n이론을 구성하는 성분의 세부적인 성분들은 직접적으로 보이지 않는다. 그러나 그 특성은 온도, 압력, 엔트로피, 열용량과 같은 눈으로 볼 수 있는 평균적인 지표들에 의해 설명된다. 그 이론의 특성에 관한 고전적인 정의는 평형 상태임을 가정하였다. 엔트로피의 고전 열역학적 정의는 최근 비평형 열역학의 영역으로까지 확대되었다. 이후 엔트로피를 포함한 열특성은 눈으로 볼 수 없는 미시세계에서의 움직임에 대한 정역학적인 관점에서 새롭게 정의되었다. 그 예로 처음에는 기체로 여겨졌지만 시간이 지난 후에 광자, 음자, 스핀과 같은 양자 역학적으로 생각되는 뉴턴 입자가 있다. 이 이론의 특성에 대한 통계학적 설명은 고전적인 열역학을 사용하여 이론의 특성을 정의하는 것이 몇몇 변수의 영향을 받는 이론의 최종적인 상태를 예측하는데 있어서 점점 신뢰할 수 없는 예측 기술이 되어감으로 인하여 필수적인 것이 되었다.\n\n열역학적 정의 \n고전적 열역학에서는 엔트로피 S의 절대적 값은 정의할 수 없고, 대신 그 상대적 변화만 정의한다. 열적 평형을 이뤄 온도가 인 계에 열 를 가하였다고 하자. 이 경우 엔트로피의 증가

# 랭체인의 api 로 결과 비교하기

In [24]:
question

['벨기에의 수도는 어디인가요?']

In [30]:
!pip install -U langchain-pinecone


   ---------------------------------------- 3/3 [langchain-pinecone]



In [31]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embedding, text_key='full_text')
docs = vector_store.similarity_search(query=question[0], k=5, namespace='wiki-ns1')
docs

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


[Document(id='7c5cc093-854b-4395-87d0-535acb11f190', metadata={'chunk_id': 11.0, 'title': '엔트로피', 'url': 'https://ko.wikipedia.org/wiki/%EC%97%94%ED%8A%B8%EB%A1%9C%ED%94%BC', 'wiki_id': '158'}, page_content='엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. 통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, 열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 라틴 대문자 S를 쓴다.\n\n정의 \n엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. 고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 성분들로 이루어져 있고, 학설의 안정성은 그러한 성분들의 평균적인 열특성으로 설명된다.\n이론을 구성하는 성분의 세부적인 성분들은 직접적으로 보이지 않는다. 그러나 그 특성은 온도, 압력, 엔트로피, 열용량과 같은 눈으로 볼 수 있는 평균적인 지표들에 의해 설명된다. 그 이론의 특성에 관한 고전적인 정의는 평형 상태임을 가정하였다. 엔트로피의 고전 열역학적 정의는 최근 비평형 열역학의 영역으로까지 확대되었다. 이후 엔트로피를 포함한 열특성은 눈으로 볼 수 없는 미시세계에서의 움직임에 대한 정역학적인 관점에서 새롭게 정의되었다. 그 예로 처음에는 기체로 여겨졌지만 시간이 지난 후에 광자, 음자, 스핀과 같은 양자 역학적으로 생각되는 뉴턴 입자가 있다. 이 이론의 특성에 대한 통계학적 설명은 고전적인 열역학을 사용하여 이론의 특성을 정의하는 것이 몇몇 변수의 영향을 받는 이론의 최종적인 상태를 예측하

In [71]:
# 질문의 응답 결과가 동일한지 검증
question = [
    "남양주의 위치는?",
    "우크라이나의 대표적인 산의 이름은 무엇인가요?",
    "한국을 대표하는 음식은 무엇인가요?"
]

In [72]:
emb_question2 = embedding.embed_documents(question)
emb_question2

[[-0.04113133251667023,
  0.04525133594870567,
  0.04092533513903618,
  0.049474336206912994,
  -0.005832375027239323,
  -0.021526999771595,
  -0.004519124981015921,
  -0.040513332933187485,
  0.0023732916451990604,
  -0.06880400329828262,
  -0.020033501088619232,
  -0.005892458371818066,
  -0.01903783343732357,
  -0.011742000468075275,
  -0.0053431252017617226,
  -0.028187667950987816,
  -0.03828166797757149,
  -0.012935083359479904,
  0.058263666927814484,
  0.02640233375132084,
  0.02037683315575123,
  -0.03642766922712326,
  -0.013673250563442707,
  0.009579000063240528,
  0.04463333263993263,
  -0.030385000631213188,
  -0.009149833582341671,
  -0.0030148958321660757,
  -0.003765937639400363,
  -0.015329834073781967,
  -0.03560366854071617,
  -0.0373203344643116,
  0.04604100063443184,
  -0.028050333261489868,
  0.006519041955471039,
  0.02657400071620941,
  0.06787700206041336,
  -0.03433333337306976,
  -0.02288316749036312,
  0.04223000258207321,
  0.012411500327289104,
  -0.0030

# Answer

In [73]:
for q in question:
    docs = vector_store.similarity_search(
        query=q,
        k=5,
        namespace="wiki-ns1"
    )
    print("질문:", q)
    for i, doc in enumerate(docs, start=1):
        print(f"결과 {i}: {doc.page_content[:100]}...")  # 청크 내용 일부 출력

    print("=" * 80)

질문: 남양주의 위치는?
결과 1: 엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. 통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태...
결과 2: 엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. 통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태...
결과 3: 대한민국 제16대 대통령 선거는 2002년 12월 19일 목요일 치뤄진 대통령 선거로, 21세기에 처음으로 치뤄진 대한민국 대통령 선거이다. 제15대 김대중 대통령의 차기 대통령을...
결과 4: 중국의 역사(中國史, )에 대한 최초의 기록은 기원전 1250년 무정의 통치기인 상나라(기원전 1600~1046년 경)로 거슬러 올라간다. 황하 문명은 여러 다른 문명의 영향을 받...
결과 5: 지구과학(地球科學, )은 행성인 지구와 그 주위의 천체를 연구하는 학문들을 묶어 부르는 이름이다. 지구의 환경은 크게 육지, 바다, 대기로 나누어지며, 이러한 환경들은 각각 지구과...
질문: 우크라이나의 대표적인 산의 이름은 무엇인가요?
결과 1: 우크라이나()는 동유럽 국가다. 남쪽과 남동쪽으로는 흑해와 아조프해, 동쪽과 북동쪽으로는 러시아, 북쪽과 북서쪽으로는 벨라루스, 서쪽으로는 폴란드, 슬로바키아, 헝가리, 남서쪽으로...
결과 2: 우크라이나()는 동유럽 국가다. 남쪽과 남동쪽으로는 흑해와 아조프해, 동쪽과 북동쪽으로는 러시아, 북쪽과 북서쪽으로는 벨라루스, 서쪽으로는 폴란드, 슬로바키아, 헝가리, 남서쪽으로...
결과 3: 우크라이나()는 동유럽 국가다. 남쪽과 남동쪽으로는 흑해와 아조프해, 동쪽과 북동쪽으로는 러시아, 북쪽과 북서쪽으로는 벨라루스, 서쪽으로는 폴란드, 슬로바키아, 헝가리, 남서쪽으로...
결과 4: 우크라이나()는 동유럽 국가다. 남쪽과 남동쪽으로는 흑해와 아조프해, 동쪽과 북동쪽으로는 러시아, 북쪽과 북서쪽으로는 벨라루스

# Augmentation

In [ ]:
# retrieval
country = '벨기에'
query = f"{country}의 여행을 위한 관광지와 문화"
docs = vector_store.similarity_search(query=query, k=5, namespace='wiki-ns1')

In [56]:
# augmentation context
context = ""
for result in docs:
    context += result.page_content

In [59]:
context

'아쿠타가와 류노스케(, 1892년 3월 1일~1927년 7월 24일)는 일본의 소설가이다. 호는 징강당주인(澄江堂主人)이며 하이쿠 작가로서의 호는 가키(我鬼)이다.\n\n그의 작품은 대부분이 단편 소설이다. 「참마죽」, 「덤불 속」, 「지옥변」 등 주로 일본의 《곤자쿠 이야기집》·《우지슈이 이야기》 등 전통적인 고전들에서 제재를 취하였다. 또한 「거미줄(원제: 蜘蛛の糸)」, 「두자춘(杜子春)」 등 어린이를 위한 작품도 남겼으며, 예수를 학대한 유대인이 예수가 세상에 다시 올 때까지 방황한다는 상상력을 발휘한 「방황하는 유대인」도 있다.\n\n생애\n\n유년 시절 \n1892년(메이지 25년) 3월 1일 도쿄에서 우유 판매업자였던 아버지 니하라 도시조(新原敏三)와 어머니 후쿠(フク) 사이의 아들로 태어났다(아쿠타가와라는 성은 원래 그의 어머니쪽 성씨였다). 이때 태어난 시간이 공교롭게도 진년(辰年) 진월(辰月) 진일(辰日) 진시(辰時)였기 때문에 \'용(龍)\' 자를 이름에 넣어 류노스케(龍之介)라 짓게 되었다고 전하나, 실제 그가 태어난 1892년 3월 1일은 간지로는 임진년·임인월·임진일에 해당하며, 출생 시각에 대해서는 자료가 없기 때문에 확실한 것이 없다. 이름도 호적상으로는 \'龍之介\'이지만 그가 양자로 들어갔던 아쿠타가와 집안이나 졸업한 학교의 명단 등의 문서에는 \'龍之助\'로 되어 있다(아쿠타가와 자신은 \'龍之助\' 표기를 싫어했다).\n\n가족 \n원래 류노스케의 위로는 하쓰(はつ)와 히사(ひさ)라는 두 명의 누나가 있었는데, 큰누나였던 하쓰는 류노스케가 태어나기 1년 전에 여섯 살의 나이로 요절했고, 어머니는 그 충격으로 정신장애를 겪어서 류노스케를 양육할 수 없었다. 생후 7개월 된 류노스케는 도쿄시 혼죠구 고이즈미쵸에 있던 외가 아쿠타가와 집안에 맡겨졌고, 백모 후키(フキ)가 양육을 맡았다. 11살 때인 1902년에 어머니가 끝내 사망하자 이듬해에 그는 외삼촌으로 도쿄시의 토목과장을 지내기도 했던 아쿠타가와 미치아키(芥川道章)의 양자가

In [57]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000017FBDFABF20>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000017FBDE39460>, root_client=<openai.OpenAI object at 0x0000017FBDF7CEF0>, root_async_client=<openai.AsyncOpenAI object at 0x0000017FBE0213A0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [58]:
from langchain.prompts import ChatPromptTemplate

# 대화식 프롬프트 작성
chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', """당신은 베테랑 여행 가이드입니다. 
         고객 맞춤 여행일정 수립을 도와드립니다. 
         제한된 정보라도 최대한 활용해서 도움을 주세요."""), 
        ('human', """저는 {country}로 여행을 계획중입니다. 
         아래 context에서 {country} 관련 정보를 찾아 여행 조언을 해주세요'
         context :
         {context}"""), # 사용자 prompt
    ]
)
chat_template

ChatPromptTemplate(input_variables=['context', 'country'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 베테랑 여행 가이드입니다. \n         고객 맞춤 여행일정 수립을 도와드립니다. \n         제한된 정보라도 최대한 활용해서 도움을 주세요.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'country'], input_types={}, partial_variables={}, template="저는 {country}로 여행을 계획중입니다. \n         아래 context에서 {country} 관련 정보를 찾아 여행 조언을 해주세요'\n         context :\n         {context}"), additional_kwargs={})])

In [60]:
prompt = chat_template.format_messages(country=country, context=context)
print(prompt)

[SystemMessage(content='당신은 베테랑 여행 가이드입니다. \n         고객 맞춤 여행일정 수립을 도와드립니다. \n         제한된 정보라도 최대한 활용해서 도움을 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='저는 벨기에로 여행을 계획중입니다. \n         아래 context에서 벨기에 관련 정보를 찾아 여행 조언을 해주세요\'\n         context :\n         아쿠타가와 류노스케(, 1892년 3월 1일~1927년 7월 24일)는 일본의 소설가이다. 호는 징강당주인(澄江堂主人)이며 하이쿠 작가로서의 호는 가키(我鬼)이다.\n\n그의 작품은 대부분이 단편 소설이다. 「참마죽」, 「덤불 속」, 「지옥변」 등 주로 일본의 《곤자쿠 이야기집》·《우지슈이 이야기》 등 전통적인 고전들에서 제재를 취하였다. 또한 「거미줄(원제: 蜘蛛の糸)」, 「두자춘(杜子春)」 등 어린이를 위한 작품도 남겼으며, 예수를 학대한 유대인이 예수가 세상에 다시 올 때까지 방황한다는 상상력을 발휘한 「방황하는 유대인」도 있다.\n\n생애\n\n유년 시절 \n1892년(메이지 25년) 3월 1일 도쿄에서 우유 판매업자였던 아버지 니하라 도시조(新原敏三)와 어머니 후쿠(フク) 사이의 아들로 태어났다(아쿠타가와라는 성은 원래 그의 어머니쪽 성씨였다). 이때 태어난 시간이 공교롭게도 진년(辰年) 진월(辰月) 진일(辰日) 진시(辰時)였기 때문에 \'용(龍)\' 자를 이름에 넣어 류노스케(龍之介)라 짓게 되었다고 전하나, 실제 그가 태어난 1892년 3월 1일은 간지로는 임진년·임인월·임진일에 해당하며, 출생 시각에 대해서는 자료가 없기 때문에 확실한 것이 없다. 이름도 호적상으로는 \'龍之介\'이지만 그가 양자로 들어갔던 아쿠타가와 집안이나 졸업한 학교의 명단 등의 문서에는 \'龍之助\'로 되어 있다(아쿠타가와 자신은 \'龍之助\' 표기를 싫어했다).\n\n

In [61]:
response = llm.invoke(prompt)
response.content

"벨기에 여행을 계획 중이시군요! 다음은 벨기에에서의 여행을 더욱 즐겁고 기억에 남게 해줄 몇 가지 제안입니다.\n\n### 1. 주요 도시 및 관광지\n- **브뤼셀**: 벨기에의 수도로, **그랑 플라스**(Grand Place), **아토미움**(Atomium), **맨넨픽**(Manneken Pis) 등을 방문하세요. 특히 그랑 플라스는 유네스코 세계문화유산으로 등록되어 있으며, 아름다운 건축물이 많습니다.\n  \n- **브루게**: 중세의 아름다움을 간직한 도시로, 운하를 따라보는 보트 크루즈를 해보세요. **시장 광장**(Markt)과 **브루게 종탑**(Belfry of Bruges)도 놓치지 마세요.\n\n- **겐트**: 개성 있는 건축물과 활기찬 분위기의 이 도시는 **그르트 시티**(Gravensteen) 성, **산 미하엘 다리** (St. Michael's Bridge) 등이 유명합니다. 현지의 다양한 카페와 음식도 즐길 수 있습니다.\n\n### 2. 음식 추천\n- **벨기에 초콜릿**: 유명한 초콜릿 가게를 방문해보세요. 고급 초콜릿을 실컷 맛볼 수 있습니다.\n  \n- **와플**: 브뤼셀 와플과 리에주 와플 모두 즐길 수 있습니다. 각각의 다른 식감과 맛을 경험해보세요.\n\n- **맥주**: 벨기에는 맥주가 유명합니다. 다양한 종류의 벨기에 맥주를 맛볼 수 있는 바나 양조장을 방문해보세요.\n\n### 3. 문화 체험\n- **켈러브»**: 벨기에의 전통 음악과 춤 등을 감상할 수 있는 공연이나 축제를 찾아보세요. 현지인들과의 교류도 좋은 경험입니다.\n\n### 4. 추천일정\n- **1일차**: 브뤼셀 도착 후 그랑 플라스, 아토미움 방문, 브뤼셀 와플 맛보기.\n- **2일차**: 브루게로 당일치기 여행, 운하 보트 투어 후 중세 도시 탐방.\n- **3일차**: 겐트 탐방, 그르트 시티 성 방문 후 현지 음식점에서 저녁.\n  \n### 5. 교통\n- 벨기에는 대중교통이 잘 되어 있어, 기차를 이용하면 도시 간 이